# Loading dataset

* y : (N,) discrete for classification, real values for regression
* x : (N, D, tn) input multivariate time series data with dimension. 
  * N is number of data cases, D is the dimension of sparse and irregularly sampled time series and tn is the union of observed time stamps in all the dimension for a data case n. Since each tn is of variable length, we pad them with zeros to have an array representation.

* m : (N, D, tn) where m[i,j,k] = 0 means that x[i,j,k] is not observed.
* T : (N, D, tn) represents the actual time stamps of observation;

In [11]:
import pickle
import copy
import numpy as np

with open('combined_vitals_data.pkl', 'rb') as file:
    vitals = pickle.load(file)

In [13]:
vitals

array([['SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (646,220277) ORDER BY charttime',
        'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (211,220045) ORDER BY charttime',
        'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (618,615,220210,224690) ORDER BY charttime',
        ...,
        'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (807,811,1529,3745,3744,225664,220621,226537) ORDER BY charttime',
        'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (780,860,1126,1673,3839,4202,4753,6003,220274,220734,223830,228243) ORDER BY charttime',
        'SELECT charttime, VALUE FROM outputevents WHERE hadm_id = 100001 AND itemid IN (43647,43053,43171,43173,43333,43347,43348,43355,43365,43373,43374,43379,43380,43431,43519,43522,43537,43576,43583,43589,43638,43654,43811,43812,43856,44706,45304,227519) ORDER BY 

In [2]:
with open('adm_type_los_mortality.p', 'rb') as file:
    adm_info = pickle.load(file)

In [8]:
adm_info

[(165315, 'EMERGENCY', Decimal('27'), 0),
 (152223, 'ELECTIVE', Decimal('131'), 0),
 (124321, 'EMERGENCY', Decimal('162'), 0),
 (161859, 'EMERGENCY', Decimal('68'), 0),
 (129635, 'EMERGENCY', Decimal('84'), 0),
 (197661, 'EMERGENCY', Decimal('167'), 0),
 (134931, 'NEWBORN', Decimal('64'), 0),
 (162569, 'ELECTIVE', Decimal('128'), 0),
 (104557, 'URGENT', Decimal('120'), 0),
 (128652, 'EMERGENCY', Decimal('183'), 1),
 (175413, 'ELECTIVE', Decimal('460'), 0),
 (176176, 'EMERGENCY', Decimal('85'), 0),
 (115799, 'EMERGENCY', Decimal('47'), 0),
 (144319, 'EMERGENCY', Decimal('62'), 0),
 (166707, 'ELECTIVE', Decimal('244'), 0),
 (182104, 'EMERGENCY', Decimal('198'), 0),
 (122659, 'EMERGENCY', Decimal('305'), 0),
 (165660, 'ELECTIVE', Decimal('241'), 0),
 (188670, 'EMERGENCY', Decimal('122'), 0),
 (185910, 'EMERGENCY', Decimal('611'), 0),
 (106266, 'NEWBORN', Decimal('234'), 0),
 (101757, 'ELECTIVE', Decimal('653'), 0),
 (174486, 'EMERGENCY', Decimal('678'), 0),
 (145674, 'EMERGENCY', Decimal(

In [3]:
adm_id = [record[0] for record in adm_info]
adm_id_needed = [record[0] for record in adm_info if record[2] >= 48]

In [4]:
vitals_dict = {}
for i in range(len(adm_id)):
    vitals_dict[adm_id[i]] = vitals[i]

In [5]:
vitals = [vitals_dict[x] for x in adm_id_needed]

In [ ]:
label = [rec[3] for x in adm_id_needed for rec in adm_info if x == rec[0]]

# Trim lossing

In [ ]:
# Original code: https://github.com/mlds-lab/interp-net/blob/master/src/mimic_preprocessing.py#L25

import numpy as np
from concurrent.futures import ProcessPoolExecutor

def trim_los_parallel(data_chunk, length_of_stay):
    num_features = 12  # final features (excluding EtCO2)
    max_length = 2881  # maximum length of time stamp
    a = np.full((len(data_chunk), num_features, max_length), -100, dtype=float)  # initialize array with -100 (missing data)
    timestamps = []

    for i in range(len(data_chunk)):
        # Process temperature conversion in a vectorized way
        if data_chunk[i][7]:
            temp_array = np.array([elem[1] for elem in data_chunk[i][7] if elem[1] is not None])
            data_chunk[i][6] += [(elem[0], temp * 1.8 + 32) for elem, temp in zip(data_chunk[i][7], temp_array)]

        # Combine data[9] with data[10] and data[11]
        data_chunk[i][9].extend(data_chunk[i][10] + data_chunk[i][11])

        # Remove unwanted elements (EtCO2 data)
        del data_chunk[i][5:7]
        del data_chunk[i][8]

        # Collect unique timestamps across all features
        all_timestamps = sorted(set([elem[0] for j in range(num_features) for elem in data_chunk[i][j]]))

        # Extract first 48-hour data
        first_ts = all_timestamps[0] if all_timestamps else None
        TS = [ts for ts in all_timestamps if (ts - first_ts).total_seconds() / 3600 <= length_of_stay]

        timestamps.append(TS)

        for j in range(num_features):
            feature_data = data_chunk[i][j]
            feature_dict = {entry[0]: entry[1] for entry in feature_data}  # Convert list to dictionary for fast lookup

            for k, ts in enumerate(TS):
                if ts in feature_dict:
                    value = feature_dict[ts]
                    if value is None or value in ('Other/Remarks', 'Comment'):
                        a[i, j, k] = -100
                    elif value in ('Normal <3 secs', 'Normal <3 Seconds', 'Brisk'):
                        a[i, j, k] = 1
                    elif value in ('Abnormal >3 secs', 'Abnormal >3 Seconds', 'Delayed'):
                        a[i, j, k] = 2
                    else:
                        a[i, j, k] = value
                else:
                    a[i, j, k] = -100  # missing data

    return a, timestamps

# Function to split the data and run in parallel
def run_trim_los_in_parallel(data, length_of_stay, num_workers=4):
    # Split data into chunks for parallel processing
    chunk_size = len(data) // num_workers
    data_chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

    results = []
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(trim_los_parallel, chunk, length_of_stay) for chunk in data_chunks]
        for future in futures:
            results.append(future.result())

    # Combine results from all chunks
    all_a = np.concatenate([result[0] for result in results], axis=0)
    all_timestamps = sum([result[1] for result in results], [])
    
    return all_a, all_timestamps

# Example usage
hours_look_ahead = 48
num_workers = 15  # Use the available 15 cores for parallel processing
vitals, timestamps = run_trim_los_in_parallel(vitals, hours_look_ahead, num_workers)


# Fixing input format

Return the input in the proper format

* x: observed values
* M: masking, 0 indicates missing values
* delta: time points of observation

In [ ]:
def fix_input_format(x, T):
    """Return the input in the proper format
    x: observed values
    M: masking, 0 indicates missing values
    delta: time points of observation
    """
    timestamp = 200
    num_features = 12

    # trim time stamps higher than 200
    for i in range(len(T)):
        if len(T[i]) > timestamp:
            T[i] = T[i][:timestamp]

    x = x[:, :, :timestamp]
    M = np.zeros_like(x)
    delta = np.zeros_like(x)
    print(x.shape, len(T))

    for t in T:
        for i in range(1, len(t)):
            t[i] = (t[i] - t[0]).total_seconds()/3600.0
        if len(t) != 0:
            t[0] = 0

    # count outliers and negative values as missing values
    # M = 0 indicates missing value
    # M = 1 indicates observed value
    # now since we have mask variable, we don't need -100
    M[x > 500] = 0
    x[x > 500] = 0.0
    M[x < 0] = 0
    x[x < 0] = 0.0
    M[x > 0] = 1

    for i in range(num_features):
        for j in range(x.shape[0]):
            for k in range(len(T[j])):
                delta[j, i, k] = T[j][k]

    return x, M, delta


x, M, delta = fix_input_format(vitals, timestamps)

# Mean inputation

In [ ]:
def mean_imputation(vitals, mask):
    """For the time series missing entirely, our interpolation network 
    assigns the starting point (time t=0) value of the time series to 
    the global mean before applying the two-layer interpolation network.
    In such cases, the first interpolation layer just outputs the global
    mean for that channel, but the second interpolation layer performs 
    a more meaningful interpolation using the learned correlations from
    other channels."""
    counts = np.sum(np.sum(mask, axis=2), axis=0)
    mean_values = np.sum(np.sum(vitals*mask, axis=2), axis=0)/counts
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if np.sum(mask[i, j]) == 0:
                mask[i, j, 0] = 1
                vitals[i, j, 0] = mean_values[j]
    return


mean_imputation(x, M)

In [ ]:
def hold_out(mask, perc=0.2):
    """To implement the autoencoder component of the loss, we introduce a set
    of masking variables mr (and mr1) for each data point. If drop_mask = 0,
    then we removecthe data point as an input to the interpolation network,
    and includecthe predicted value at this time point when assessing
    the autoencoder loss. In practice, we randomly select 20% of the
    observed data points to hold out from
    every input time series."""
    drop_mask = np.ones_like(mask)
    drop_mask *= mask
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            count = np.sum(mask[i, j], dtype='int')
            if int(0.20*count) > 1:
                index = 0
                r = np.ones((count, 1))
                b = np.random.choice(count, int(0.20*count), replace=False)
                r[b] = 0
                for k in range(mask.shape[2]):
                    if mask[i, j, k] > 0:
                        drop_mask[i, j, k] = r[index]
                        index += 1
    return drop_mask

drop_mask=hold_out(M)

In [ ]:
x = np.concatenate((x, M, delta, drop_mask), axis=1)

In [ ]:
print(x.shape)

In [ ]:
y= np.array(label)
print(y.shape)

In [ ]:
np.savez('preprocessed_data.npz', array1=x, array2=y)

# Acknowledgement

* https://github.com/mlds-lab/interp-net/blob/master/src/mimic_preprocessing.py#L25